In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
# Computational modules
import dask
import xarray as xr
import gcsfs
import datetime
# Maps   
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import rioxarray
import geopandas as gpd
from shapely.geometry import box, mapping
import dateutil
from cartopy.util import add_cyclic_point
import import_ipynb
import os

# Local files and utility functions
sys.path.append("./subroutines/")
import pices
import tables
import pytz
import pvlib

# Import other notebooks
import albedo
import albedo_tools
import regrid_cmip6
import seasonal_average

import xesmf as xe
np.seterr(divide='ignore', invalid='ignore')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

get_ipython().run_line_magic("matplotlib", "inline")
plt.rcParams["figure.figsize"] = 12, 6
get_ipython().run_line_magic("config", "InlineBackend.figure_format = 'retina'")

importing Jupyter notebook from albedo.ipynb
importing Jupyter notebook from albedo_tools.ipynb
importing Jupyter notebook from regrid_cmip6.ipynb
importing Jupyter notebook from seasonal_average.ipynb


In [2]:
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:43359 Dashboard: /user/trondkr/proxy/8787/status,Cluster Workers: 4 Cores: 16 Memory: 63.33 GB


In [3]:
class Config_pices():
    df = pd.read_csv("https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv")
    fs = gcsfs.GCSFileSystem(token="anon", access="read_only")
    grid_labels = ["gn"]  # Can be gr=grid rotated, or gn=grid native
    member_ids = ["r1i1p1f1"]  #
    experiment_ids = ["ssp585"]  #'abrupt-4xCO2',
    source_ids = ["ACCESS-ESM1-5"] #"CanESM5"] #"MPI-ESM1-2-LR"]
    variable_ids = ["uas","vas","chl","clt","sithick", "siconc", "sisnthick", "sisnconc"]
    table_ids = ["Amon","Amon","Omon","Amon","SImon","SImon","SImon","SImon"]  # Amon=atmospheric variables, Omon=Ocean variables, SImon=sea-ice variables
    dset_dict = {}
    start_date="1950-01-01"
    end_date="2100-01-01"
    clim_start="1961-01-01"
    clim_end="1990-01-01"
    selected_depth=0
  
# Create the object
config_pices_obj = Config_pices()
config_pices_obj = pices.get_and_organize_cmip6_data(config_pices_obj)

Running historical query on data: 
 ==> source_id=='ACCESS-ESM1-5'and table_id=='Amon' and grid_label=='gn' and experiment_id=='historical' and variable_id=='uas'

Time encoding: DatetimeIndex(['1850-01-16 12:00:00', '1850-02-15 00:00:00',
               '1850-03-16 12:00:00', '1850-04-16 00:00:00',
               '1850-05-16 12:00:00', '1850-06-16 00:00:00',
               '1850-07-16 12:00:00', '1850-08-16 12:00:00',
               '1850-09-16 00:00:00', '1850-10-16 12:00:00',
               ...
               '2014-03-16 12:00:00', '2014-04-16 00:00:00',
               '2014-05-16 12:00:00', '2014-06-16 00:00:00',
               '2014-07-16 12:00:00', '2014-08-16 12:00:00',
               '2014-09-16 00:00:00', '2014-10-16 12:00:00',
               '2014-11-16 00:00:00', '2014-12-16 12:00:00'],
              dtype='datetime64[ns]', name='time', length=1980, freq=None) - datetime64[ns]
Running projections query on data: 
 ==> source_id=='ACCESS-ESM1-5'and table_id=='Amon' and member_

In [4]:
# Following Roland Seferian, equation 3
# https://www.geosci-model-dev.net/11/321/2018/gmd-11-321-2018.pdf

def calculate_alpha_dir(n_lambda,µ):
    a = np.sqrt(1.0 - (1.0 - µ**2)/n_lambda**2)
    b = ((a-n_lambda*µ)/(a+n_lambda*µ))**2    
    c = ((µ-n_lambda*a)/(µ+n_lambda*a))**2

    return 0.5*(b+c)

In [5]:
def calculate_diffuse_reflection(n_λ,σ):
    # Diffuse albedo from Jin et al., 2006 (Eq 5b) 
    return -0.1479 + 0.1502*n_λ-0.0176*n_λ*σ

In [6]:
def surface_roughness(µ, σ):
    # Surface roughness following Jin et al. 2014 equation 4
    # This rougness parameter determines the Fresnel refraction 
    # index from flat surface
    return (0.0152-1.7873*µ + 6.8972*(µ**2)-8.5778*(µ**3)+ 4.071*σ-7.6446*µ*σ) * np.exp(0.1643-7.8409*µ-3.5639*µ**2-2.3588*σ+10.054*µ*σ)

In [7]:
def calculate_direct_reflection(n_λ,µ,σ):
    # Direct reflection following Jin et al. 2014 equation 1
    f_0 = calculate_alpha_dir(1.34,µ)
    f_λ = calculate_alpha_dir(n_λ,µ)
    
    return f_λ-(surface_roughness(µ, σ)*f_λ/f_0)

In [8]:
def calculate_direct_reflection_from_chl(λ, chl, alpha_chl, alpha_w, beta_w, σ, µ, alpha_direct):
   
    rw=0.48168549-0.014894708*σ-0.20703885*σ**2
 
    # Determine absorption and backscattering
    # coefficients to determine reflectance below the surface (Ro) once for all
    # Backscattering by chlorophyll:
    a_bp = 0.06*alpha_chl*np.exp(np.log(chl)*0.65) + 0.2*(0.00635+0.06*(np.exp(np.log(chl)*0.65))*np.exp(0.014*(440.0-λ)))
   
    # Backscattering of biological pigment (b_chl) with λ expressed here in nm and [Chl] in mg m−3. This
    # formulation is valid for [Chl] ranging between 0.02 and 2 mg m−3 (Morel and Maritorena (2001))
    # Equation 12 Roland Seferian, 2018
    b_chl=(0.416*np.exp(0.766*np.log(chl)))*(0.002+0.01*(0.5-0.25*np.log(chl))*np.exp((0.5*(np.log(chl)-0.3))*np.log(λ/550.0)))
                                                                                 
    # # Use Morel 91 formula to compute the direct reflectance below the surface (Morel-Gentili(1991), Eq (12))
    n=0.5*beta_w/(0.5*beta_w + b_chl)
   
    # Equation 11 Roland Seferian, 2018
    beta = 0.6279-0.2227*n-0.0513*n**2 +(0.2465*n - 0.3119)*µ
    
    # Equation 10 Roland Seferian, 2018
    R0 = beta * (0.5*beta_w + b_chl)/(alpha_w + a_bp)
   
    # Water leaving albedo, equation 8 Roland Seferian, 2018
    return (R0*(1.0-rw)/(1-rw*R0)) 

In [9]:
def calculate_diffuse_reflection_from_chl(λ, chl, alpha_chl, alpha_w, beta_w, σ, alpha_direct):
    #  In the case of ocean interior reflectance for direct incoming radiation it depends on µ = cos(θ) whereas in the
    # case of ocean interior reflectance for diffuse µ = 0.676. This value is considered an effective angle of incoming radiation of 47.47◦
    # according to Morel and Gentili (1991). Hence
    return calculate_direct_reflection_from_chl(λ, chl, alpha_chl, alpha_w, beta_w, σ, np.arccos(0.676), alpha_direct)

In [10]:
def whitecap(wind):
    # Whitecap effect as defined by Salisbury et al. 2014. NOTE that the value in paper is in percent
    # so we use the ratio instead (/100.) 
    # Salisbury, D. J., Anguelova, M. D., and Brooks, I. M.: Global Distribution and Seasonal 
    # Dependence of Satellite-based Whitecap Fraction
    #
    # Whitecaps are the surface manifestation of bubble plumes, created when 
    # surface gravity waves break and entrain air into the water column. 
    # They enhance air-sea exchange, introducing physical processes different from 
    # those operating at the bubble-free water surface. 
  
    return 0.000397*(np.exp(1.59*np.log(wind)))

In [11]:
def calculate_spectral_and_broadband_OSA(wind,alpha_wc,alpha_direct,alpha_diffuse,alpha_direct_chl,alpha_diffuse_chl,solar_energy):
    wc = whitecap(wind)
    OSA=np.zeros((1,2))
    OSA_direct = (alpha_direct + alpha_direct_chl) * (1-wc) + wc*alpha_wc
    OSA_diffuse = (alpha_diffuse + alpha_diffuse_chl) * (1-wc) + wc*alpha_wc
    
    # Integrate across all wavelengths 200-4000nm at 10 nm wavelength bands and then weight by the solar energy at each band.
    # The solar energy is dimensionless with sum equal to 1 and therefore already weighted.
    OSA_direct_broadband = np.sum(OSA_direct*solar_energy)
    OSA_diffuse_broadband = np.sum(OSA_diffuse*solar_energy)
    
    OSA[0,0]=OSA_direct_broadband
    OSA[0,1]=OSA_diffuse_broadband
    return OSA

In [12]:
def calculate_OSA(µ_deg, uv, chl, wavelengths, refractive_indexes, alpha_chl, alpha_w, beta_w, alpha_wc, solar_energy):
    
    if (µ_deg<0 or µ_deg>180):
        µ_deg=0
 
    µ = np.cos(np.radians(µ_deg))
    
    # Solar zenith angle
    # wind is wind at 10 m height (m/s)
    σ = np.sqrt(0.003+0.00512*uv)
         
    # Vectorize the functions
    vec_calculate_direct_reflection=np.vectorize(calculate_direct_reflection)
    vec_calculate_diffuse_reflection=np.vectorize(calculate_diffuse_reflection)
    vec_calculate_direct_reflection_from_chl=np.vectorize(calculate_direct_reflection_from_chl)
    vec_calculate_diffuse_reflection_from_chl=np.vectorize(calculate_diffuse_reflection_from_chl)
    
    # Direct reflection
    alpha_direct = vec_calculate_direct_reflection(refractive_indexes,µ,σ)
   
    # Diffuse reflection
    alpha_diffuse = vec_calculate_diffuse_reflection(refractive_indexes,σ)

    # Reflection from chlorophyll and biological pigments
    alpha_direct_chl = vec_calculate_direct_reflection_from_chl(wavelengths, chl, alpha_chl, alpha_w, beta_w, σ, µ, alpha_direct)
   
    # Diffuse reflection interior of water from chlorophyll
    alpha_diffuse_chl = vec_calculate_diffuse_reflection_from_chl(wavelengths, chl, alpha_chl, alpha_w, beta_w, σ, alpha_direct)

    # OSA
    return calculate_spectral_and_broadband_OSA(uv,alpha_wc,alpha_direct,alpha_diffuse,alpha_direct_chl,alpha_diffuse_chl,solar_energy)

In [13]:
def setup_parameters():
    df=pd.read_csv("data/Wavelength/Fresnels_refraction.csv", header=0, sep=";", decimal=",")
    wavelengths=df["λ"].values
    refractive_indexes=df["n(λ)"].values
    alpha_chl=df["a_chl(λ)"].values
    alpha_w=df["a_w(λ)"].values
    beta_w=df["b_w(λ)"].values
    alpha_wc=df["a_wc(λ)"].values
    solar_energy=df["E(λ)"].values
    print(df.head())
    return wavelengths, refractive_indexes, alpha_chl, alpha_w, beta_w, alpha_wc, solar_energy

### Radiation calculations
To calculate the irradiance at any given location on the planet at the exact same time of day we use the excellent [pvlib](https://github.com/pvlib/pvlib-python) Python library. This library allows us to calculate the direct and diffuse irradiance at the surface. The diffuse and direct (beam) irradiance are both corrected for cloud cover by applying the Liu Jordan equation. The approach is to sequentially calculate the following for each latitude in our grid, but for a fixed longitude position (lon=0) assuming that the solar daily cycle only varies with latitutde and time of day. The cloud cover is accounted for by first calculating the clear sky radiation and then scaling for cloud cover using the Liu Jordan equation.

In [14]:
def radiation(cloud_covers,latitude,month,hour_of_day):
    results=np.zeros((np.shape(cloud_covers)[0],3))
    #lon_180 = np.where(longitude > 180, longitude-360, longitude)
    offset=0 #int(lon_180/15.)
    when =[datetime.datetime(2006,month,15,hour_of_day,0,0,tzinfo=datetime.timezone(datetime.timedelta(hours=offset)))]
    time = pd.DatetimeIndex(when)
    sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')
    sapm_inverters = pvlib.pvsystem.retrieve_sam('cecinverter')

    module = sandia_modules['Canadian_Solar_CS5P_220M___2009_']
    inverter = sapm_inverters['ABB__MICRO_0_25_I_OUTD_US_208__208V_']
    system = {'module': module, 'inverter': inverter,
                  'surface_azimuth': 180}
    altitude=0.0
    system['surface_tilt'] = latitude
        
    
    # Some calculations are done only on greenwhich meridian line as they are identical around the globe 
    # at the same latitude. For that reason longitude is set to greenwhich meridian and do not change. The only reason 
    # to use longitude would be to have the local sun position for given time but since we calculate position at the same
    # time of the day (hour_of_day) and month (month) we can assume its the same across all longitudes,
    # and only change with latitude.
    
    longitude=0.0
    solpos = pvlib.solarposition.get_solarposition(time, latitude, longitude)

    dni_extra = pvlib.irradiance.get_extra_radiation(time)
    airmass = pvlib.atmosphere.get_relative_airmass(solpos['apparent_zenith'])

    pressure = pvlib.atmosphere.alt2pres(altitude)
    am_abs = pvlib.atmosphere.get_absolute_airmass(airmass, pressure)
    tl = pvlib.clearsky.lookup_linke_turbidity(time,latitude,longitude)

    for cloud_index,cloud_cover in enumerate(cloud_covers):
        
        # cloud cover in percentage units here
        transmittance = ((100.0 - cloud_cover) / 100.0) * 0.75
        # irrads is a DataFrame containing ghi, dni, dhi
        irrads = pvlib.irradiance.liujordan(solpos['apparent_zenith'], transmittance, am_abs)

        aoi = pvlib.irradiance.aoi(system['surface_tilt'], system['surface_azimuth'],
                                   solpos['apparent_zenith'], solpos['azimuth'])

        total_irrad = pvlib.irradiance.get_total_irradiance(system['surface_tilt'],
                                                            system['surface_azimuth'],
                                                            solpos['apparent_zenith'],
                                                            solpos['azimuth'],
                                                            irrads['dni'], irrads['ghi'], irrads['dhi'],
                                                            dni_extra=dni_extra,
                                                            model='haydavies')


       # effective_irradiance = pvlib.pvsystem.sapm_effective_irradiance(
       #    total_irrad['poa_direct'], total_irrad['poa_diffuse'],
       #    am_abs, aoi, module)
       # print("({},{}) total {} direct: {} diffuse: {}".format(latitude,longitude,total_irrad['poa_direct'][0]+total_irrad['poa_diffuse'][0],
       #                                                total_irrad['poa_direct'][0], 
       #                                                total_irrad['poa_diffuse'][0]))
       
        results[cloud_index,0]=total_irrad['poa_direct']
        results[cloud_index,1]=total_irrad['poa_diffuse']
        results[cloud_index,2]=solpos['zenith']
       
    return results

In [15]:
def create_streamplot(indata_u,indata_v,uv, lon,lat,name,nlevels=None):
    # Make data cyclic around dateline
    fig = plt.figure(figsize=(12, 12))
    crs_latlon = ccrs.PlateCarree()
    
    indata_u_cyclic, lon_cyclic = add_cyclic_point(indata_u, coord=lon)
    indata_v_cyclic, lon_cyclic = add_cyclic_point(indata_v, coord=lon)
    
    cf = ax.contourf(lon_cyclic, lat, uv, 10,
                 cmap='RdYlBu_r',
                 extend='both',
                 transform=ccrs.PlateCarree())
    
    land_110m = cfeature.NaturalEarthFeature('physical', 'land', '110m')  
    sp = ax.streamplot(lon_cyclic, lat, indata_u_cyclic, indata_v_cyclic,
                   linewidth=0.5,
                   arrowsize = 0.4,
                   density=10,
                   color='k',
                   transform=ccrs.PlateCarree())

    cb = plt.colorbar(cf,orientation='horizontal', pad=0.04, aspect=50)
    cb.ax.set_title('Wind speed [m/s]')
    ax.add_feature(land_110m, color="lightgrey")
    ax.add_feature(cfeature.COASTLINE, edgecolor="black")
    plt.show()

In [16]:
def create_plot(indata,lon,lat,name,nlevels=None,regional=False,logscale=False):
    
    proj=ccrs.PlateCarree()
    proj=ccrs.NorthPolarStereo(true_scale_latitude=70)
    ax = plt.axes(projection=proj)
    land_10m = cfeature.NaturalEarthFeature('physical', 'land', '10m')
    ax.add_feature(land_10m, color="lightgrey", edgecolor="black")
  #  if regional:
  #      land_10m = cfeature.NaturalEarthFeature('physical', 'land', '10m')
  #      ax.add_feature(land_10m, color="lightgrey", edgecolor="black")
  #      ax.set_extent([-180, 180, 45, 90])
    
    #indata_cyclic=indata
    #lon_cyclic = lon 
    indata_cyclic, lon = add_cyclic_point(indata,coord=lon)
    if nlevels is None:
        if logscale:
            from matplotlib import ticker, cm
            cs=ax.contourf(lon,lat,indata_cyclic,10,
                           transform=ccrs.PlateCarree(),
                           cmap='RdYlBu_r',locator=ticker.LogLocator(subs=range(1,5)),extend='both')
        else:
            cs=ax.contourf(lon,lat,indata_cyclic,20,transform=ccrs.PlateCarree(),cmap='RdYlBu_r',extend='both')
    else:
        cs=ax.contourf(lon,lat,indata_cyclic,nlevels,transform=ccrs.PlateCarree(),cmap='RdYlBu_r',extend='both')
    plt.title("{}".format(name))
    
 #   if regional:
 #       land_10m = cfeature.NaturalEarthFeature('physical', 'land', '10m')
 #       ax.add_feature(land_10m, color="lightgrey", edgecolor="black")
 #       ax.set_extent([-180, 180, 45, 90])
 #   else:
 #       land_110m = cfeature.NaturalEarthFeature('physical', 'land', '110m')
 #       ax.add_feature(land_110m, color="lightgrey", edgecolor="black")
    ax.add_feature(cfeature.BORDERS, linestyle=':')
        
    plt.colorbar(cs,shrink=0.5)
    plt.show()
    if not os.path.exists("Figures"): 
        os.mkdir("Figures")
    plotfilename="{}_anomaly_2050_2020.png".format(name)
 #   plt.savefig(plotfilename, dpi=150, bbox_inches='tight')

In [17]:
def create_chlorophyll_avg_for_year(year,ds_in,ds_out):
    start_date='{}-01-01'.format(year)
    end_date='{}-12-31'.format(year)
    ds_chl_2020=ds_in.sel(time=slice(start_date, end_date)) #.mean('time')
    
    year2=2050
    start_date='{}-01-01'.format(year2)
    end_date='{}-12-31'.format(year2)
    ds_chl_2050=ds_in.sel(time=slice(start_date, end_date)) 
    
    chl2020=regrid_cmip6.regrid_variable("chl",ds_chl_2020,ds_out,transpose=False) 
    chl2050=regrid_cmip6.regrid_variable("chl",ds_chl_2050,ds_out,transpose=False) 
    
    ds_2020=chl2020.to_dataset()
    ds_2050=chl2050.to_dataset()
       
    lat=ds_2020.lat.values
    lon=ds_2020.lon.values
        
    weighted_average_2020 = season_mean(ds_2020,calendar="noleap")
    weighted_average_2050 = season_mean(ds_2050,calendar="noleap")
    
    ds_diff =100*(weighted_average_2050 - weighted_average_2020)/weighted_average_2020
    chl2020=(weighted_average_2020.sel(season="MAM").chl.values)
    chl2050=(weighted_average_2050.sel(season="MAM").chl.values)
    chl2050_diff=ds_diff.sel(season="MAM").chl.values
    
    # kg/m3 to mg/m3 multiply by 1e6
    create_plot((chl2020*1.e6),lon[0,:],lat[:,0],"chl2020",nlevels=np.arange(0,5,0.2),regional=True,logscale=True)
    create_plot((chl2050*1.e6),lon[0,:],lat[:,0],"chl2050",nlevels=np.arange(0,5,0.2),regional=True,logscale=True)
    create_plot((chl2050_diff),lon[0,:],lat[:,0],"chl2050-2020",nlevels=np.arange(-101,101,1), regional=True)
   

In [18]:
def season_mean(ds, calendar='standard'):
    # Make a DataArray of season/year groups
    
    year_season = xr.DataArray(ds.time.to_index().to_period(freq='Q-NOV').to_timestamp(how='E'),
                               coords=[ds.time], name='year_season')

    # Make a DataArray with the number of days in each month, size = len(time)
    month_length = xr.DataArray(albedo_tools.get_dpm(ds.time.to_index(), calendar=calendar),
                                coords=[ds.time], name='month_length')
    # Calculate the weights by grouping by 'time.season'
    weights = month_length.groupby('time.season') / month_length.groupby('time.season').sum()

    # Test that the sum of the weights for each season is 1.0
    np.testing.assert_allclose(weights.groupby('time.season').sum().values, np.ones(4))

    # Calculate the weighted average
    return ((ds * weights).groupby('time.season').sum(dim='time'))


In [19]:
def regrid_variable(varname,ds_in,ds_out,transpose=True):
    regridder = xe.Regridder(ds_in, ds_out, 'bilinear', reuse_weights=True, periodic=True, ignore_degenerate=False)
    #regridder.clean_weight_file()
    
    regridder._grid_in = None
    regridder._grid_out = None

    if transpose:
        return regridder(ds_in[varname].T)
    else:
        return regridder(ds_in[varname])

In [48]:
def create_plots(sisnconc,sisnthick,sithick,siconc,clt,chl,rads,irradiance_water,wind,OSA,lon,lat):
    # create_streamplot(dr_out_uas,dr_out_vas,wind,lon[0,:],lat[:,0],"wind",nlevels=None)
    create_plot(wind,lon[0,:],lat[:,0],"wind",regional=True)
    create_plot(sisnconc,lon[0,:],lat[:,0],"sisnconc",regional=True)
    create_plot(sisnthick,lon[0,:],lat[:,0],"sisnthick",regional=True)
    create_plot(siconc,lon[0,:],lat[:,0],"siconc",regional=True)
    create_plot(sithick,lon[0,:],lat[:,0],"sithick",regional=True)
    create_plot(clt,lon[0,:],lat[:,0],"clouds",regional=True)
    create_plot(np.log(chl),lon[0,:],lat[:,0],"chl (np.lon)",regional=True)
    create_plot(OSA[:,:,0],lon[0,:],lat[:,0],"OSA_direct_broadband",np.arange(0.01,0.04,0.001),regional=True)
    create_plot(OSA[:,:,1],lon[0,:],lat[:,0],"OSA_diffuse_broadband",regional=True)
                 

In [ ]:
%%time
def calculate_light(config_pices_obj):
      
    selected_time=0
    wavelengths, refractive_indexes, alpha_chl, alpha_w, beta_w, alpha_wc, solar_energy = albedo.setup_parameters()
    startdate=datetime.datetime.now()
    
    regional=True
   
    southern_limit_latitude=45
    for key in config_pices_obj.dset_dict.keys():
        
        var_name = key.split("_")[0]
        model_name = key.split("_")[3]
        
        if var_name=="uas":
            
            key_v="vas"+key[3:]
            key_chl="chl"+key[3:]
            key_clt="clt"+key[3:]
            key_sisnconc="sisnconc"+key[3:]
            key_sisnthick="sisnthick"+key[3:]
            key_siconc="siconc"+key[3:]
            key_sithick="sithick"+key[3:]
            
            var_name_v = key_v.split("_")[0]
            model_name_v = key_v.split("_")[3]
            
            print("=> model: {} variable name: {}".format(key, var_name))
            print("=> model: {} variable name: {}".format(key_v, var_name_v))
          
            if model_name_v==model_name:
                
                ds_uas=config_pices_obj.dset_dict[key].isel(time=selected_time) 
                ds_vas=config_pices_obj.dset_dict[key_v].isel(time=selected_time)
                ds_chl=config_pices_obj.dset_dict[key_chl].isel(time=selected_time)
                ds_clt=config_pices_obj.dset_dict[key_clt].isel(time=selected_time)
                ds_sisnconc=config_pices_obj.dset_dict[key_sisnconc].isel(time=selected_time)
                ds_sisnthick=config_pices_obj.dset_dict[key_sisnthick].isel(time=selected_time)
                ds_siconc=config_pices_obj.dset_dict[key_siconc].isel(time=selected_time)
                ds_sithick=config_pices_obj.dset_dict[key_sithick].isel(time=selected_time)
                
                if regional:
                    ds_uas=ds_uas.sel(y=slice(southern_limit_latitude,90))
                    ds_vas=ds_vas.sel(y=slice(southern_limit_latitude,90))
                    ds_chl=ds_chl.sel(y=slice(southern_limit_latitude,90))
                    ds_clt=ds_clt.sel(y=slice(southern_limit_latitude,90))
                    ds_sisnconc=ds_sisnconc.sel(y=slice(southern_limit_latitude,90))
                    ds_sisnthick=ds_sisnthick.sel(y=slice(southern_limit_latitude,90))
                    ds_siconc=ds_siconc.sel(y=slice(southern_limit_latitude,90))
                    ds_sithick=ds_sithick.sel(y=slice(southern_limit_latitude,90))
                
                # Regrid to cartesian grid:
                # For any Amon related variables (wind, clouds), the resolution from CMIP6 models is less than 
                # 1 degree longitude x latitude. To interpolate to a 1x1 degree grid we therefore first interpolate to a 
                # 2x2 degrees grid and then subsequently to a 1x1 degree grid.
                
                ds_out_amon = xe.util.grid_2d(-180,180,2,southern_limit_latitude,90,2) 
                ds_out = xe.util.grid_2d(-180,180,1,southern_limit_latitude,90,1) #grid_global(1, 1)
                
                dr_out_uas_amon=regrid_variable("uas",ds_uas,ds_out_amon,transpose=True).to_dataset()
                dr_out_uas=regrid_variable("uas",dr_out_uas_amon,ds_out,transpose=False)
                
                dr_out_vas_amon=regrid_variable("vas",ds_vas,ds_out_amon,transpose=True).to_dataset()
                dr_out_vas=regrid_variable("vas",dr_out_vas_amon,ds_out,transpose=False)
                
                dr_out_clt_amon=regrid_variable("clt",ds_clt,ds_out_amon,transpose=True).to_dataset()
                dr_out_clt=regrid_variable("clt",dr_out_clt_amon,ds_out,transpose=False)
                dr_out_chl=regrid_variable("chl",ds_chl,ds_out,transpose=False)
                
                dr_out_sisnconc=regrid_variable("sisnconc",ds_sisnconc,ds_out,transpose=False)
                dr_out_sisnthick=regrid_variable("sisnthick",ds_sisnthick,ds_out,transpose=False)
                dr_out_siconc=regrid_variable("siconc",ds_siconc,ds_out,transpose=False)
                dr_out_sithick=regrid_variable("sithick",ds_sithick,ds_out,transpose=False)
                
                # Calculate scalar wind and organize the data arrays to be used for  given timestep (month-year)
                wind=np.sqrt(dr_out_uas**2+dr_out_vas**2).values
               
                lat=dr_out_uas.lat.values
                lon=dr_out_uas.lon.values
                
                clt=dr_out_clt.values
                chl=dr_out_chl.values
                sisnconc=dr_out_sisnconc.values
                sisnthick=dr_out_sisnthick.values
                siconc=dr_out_siconc.values
                sithick=dr_out_sithick.values
                
                m=len(wind[:,0])
                n=len(wind[0,:])
                month=6
                
                all_direct=[]
                all_OSA=[]
                for hour_of_day in range(12,13,1):
                    print("Running for hour {}".format(hour_of_day))
                 
                    calc_radiation = [dask.delayed(radiation)(clt[j,:],lat[j,0],month,hour_of_day) for j in range(m)]
                   
                    # https://github.com/dask/dask/issues/5464   
                    rad = dask.compute(calc_radiation, scheduler='processes')
                    rads=np.asarray(rad).reshape((m, n, 3))
                    
                    zr = [dask.delayed(calculate_OSA)(rads[i,j,2], wind[i,j], chl[i,j], wavelengths, refractive_indexes, 
                                                    alpha_chl, alpha_w, beta_w, alpha_wc, solar_energy) 
                                  for i in range(m) 
                                  for j in range(n)]

                    OSA = np.asarray(dask.compute(zr)).reshape((m, n, 2))
                    
                    irradiance_water = (rads[:,:,0]*OSA[:,:,0]+rads[:,:,1]*OSA[:,:,1])/(OSA[:,:,0]+OSA[:,:,1])
                    
                    print("Time to finish {} with mean OSA {}".format(datetime.datetime.now()-startdate,
                          np.mean(irradiance_water)))
                    
                    # Write to file
                    data_array=xr.DataArray(data=irradiance_water,dims={'lat':lat,'lon':lon})
                    if not os.path.exists("ncfiles"):
                        os.mkdir("ncfiles")
                    data_array.to_netcdf("ncfiles/irradiance.nc")

                    create_plots(sisnconc,sisnthick,sithick,siconc,clt,chl,rads,irradiance_water,wind,OSA,lon,lat)
                    
calculate_light(config_pices_obj)

       λ      E(λ)  n(λ)  a_chl(λ)  a_w(λ)  b_w(λ)  a_wc(λ)
0  200.0  0.000054  1.45     0.775   3.070  0.1510      0.0
1  210.0  0.000206  1.44     0.752   1.990  0.1190      0.0
2  220.0  0.000349  1.42     0.730   1.310  0.0995      0.0
3  230.0  0.000384  1.41     0.708   0.928  0.0820      0.0
4  240.0  0.000292  1.40     0.685   0.718  0.0685      0.0
=> model: uas_ssp585_gn_ACCESS-ESM1-5_r1i1p1f1 variable name: uas
=> model: vas_ssp585_gn_ACCESS-ESM1-5_r1i1p1f1 variable name: vas
Reuse existing file: bilinear_192x37_23x180_peri.nc
Reuse existing file: bilinear_23x180_45x360_peri.nc
Reuse existing file: bilinear_192x36_23x180_peri.nc
Reuse existing file: bilinear_23x180_45x360_peri.nc
Reuse existing file: bilinear_192x37_23x180_peri.nc


/opt/conda/lib/python3.7/site-packages/xesmf/backend.py:36: UserWarning: Input array is not F_CONTIGUOUS. Will affect performance.
  warnings.warn("Input array is not F_CONTIGUOUS. "
/opt/conda/lib/python3.7/site-packages/xesmf/backend.py:36: UserWarning: Input array is not F_CONTIGUOUS. Will affect performance.
  warnings.warn("Input array is not F_CONTIGUOUS. "
/opt/conda/lib/python3.7/site-packages/xesmf/backend.py:36: UserWarning: Input array is not F_CONTIGUOUS. Will affect performance.
  warnings.warn("Input array is not F_CONTIGUOUS. "


Reuse existing file: bilinear_23x180_45x360_peri.nc
Reuse existing file: bilinear_87x360_45x360_peri.nc
Reuse existing file: bilinear_87x360_45x360_peri.nc


/opt/conda/lib/python3.7/site-packages/xesmf/backend.py:36: UserWarning: Input array is not F_CONTIGUOUS. Will affect performance.
  warnings.warn("Input array is not F_CONTIGUOUS. "
/opt/conda/lib/python3.7/site-packages/xesmf/backend.py:36: UserWarning: Input array is not F_CONTIGUOUS. Will affect performance.
  warnings.warn("Input array is not F_CONTIGUOUS. "
/opt/conda/lib/python3.7/site-packages/xesmf/backend.py:36: UserWarning: Input array is not F_CONTIGUOUS. Will affect performance.
  warnings.warn("Input array is not F_CONTIGUOUS. "
/opt/conda/lib/python3.7/site-packages/xesmf/backend.py:36: UserWarning: Input array is not F_CONTIGUOUS. Will affect performance.
  warnings.warn("Input array is not F_CONTIGUOUS. "
/opt/conda/lib/python3.7/site-packages/xesmf/backend.py:36: UserWarning: Input array is not F_CONTIGUOUS. Will affect performance.
  warnings.warn("Input array is not F_CONTIGUOUS. "


Reuse existing file: bilinear_87x360_45x360_peri.nc
Reuse existing file: bilinear_87x360_45x360_peri.nc
Reuse existing file: bilinear_87x360_45x360_peri.nc
Running for hour 12
